In [ ]:
!pip install -q pyomo
!pip install pyomo
!apt-get install -y -qq coinor-cbc
!apt-get install -y glpk-utils
!apt install glpk-utils
!pip install glpk

Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

In [ ]:
from pyomo.environ import *
import pyomo.opt as po
import random
import numpy as np

# --------------------- Helper Function ---------------------
def g_function(x):
    Dalet = 0.5
    FinalValue = 1.0
    return FinalValue * (1 - np.exp(-Dalet * x))

# --------------------- Random Seed ---------------------
random.seed(42)
np.random.seed(42)

# --------------------- Index Sets ---------------------
Model = ConcreteModel()
i_size = 10; j_size = 15; h_size = 3; q_size = 10;
v_size = 3; g_size = 15; k_size = 3; l_size = 12;
p_size = 3; n_size = 20; m_size = 3; c_size = 40;
r_size = 11; s_size = 100;


Model.i = RangeSet(1, i_size)
Model.j = RangeSet(1, j_size)
Model.h = RangeSet(1, h_size)
Model.q = RangeSet(1, q_size)
Model.v = RangeSet(1, v_size)
Model.g = RangeSet(1, g_size)
Model.k = RangeSet(1, k_size)
Model.l = RangeSet(1, l_size)
Model.p = RangeSet(1, p_size)
Model.n = RangeSet(1, n_size)
Model.m = RangeSet(1, m_size)
Model.c = RangeSet(1, c_size)
Model.r = RangeSet(1, r_size)
Model.s = RangeSet(1, s_size)

# --------------------- All Parameters Dictionary ---------------------
parameters = {
    'fc': fc, 'ff': ff, 'fd': fd,
    'v1': v1, 'v2': v2, 'v3': v3, 'v4': v4,
    'v5': v5, 'v6': v6, 'v7': v7,
    'cc': cc, 'cp': cp, 'cq': cq, 'cr': cr,
    'ca1': ca1, 'ca2': ca2, 'ca3': ca3, 'ca4': ca4,
    't': t, 'wp': wp, 'bp': bp, 'wb': wb,
    'cs': cs, 'cf': cf,
    'bs_hat': bs_hat, 'bs_upper': bs_upper, 'bs_lower': bs_lower,
    'ps': ps, 'pd': pd, 'pe': pe,
    'dd_hat': dd_hat, 'dd_upper': dd_upper, 'dd_lower': dd_lower,
    'de_hat': de_hat, 'de_upper': de_upper, 'de_lower': de_lower,
    'b': b, 'g_b': g_b,
    'Lambda': Lambda,
    'BigM': BigM,
}

In [ ]:
def DSP(parameters,X_dict,Y_dict,Z_dict,landa,theta,ss,it):
  global i_size, j_size, h_size, q_size, v_size, g_size, k_size, l_size, p_size, n_size, m_size, c_size, r_size, s_size
  DSP_Model = ConcreteModel()

  DSP_Model.i = RangeSet(1, i_size)
  DSP_Model.j = RangeSet(1, j_size)
  DSP_Model.h = RangeSet(1, h_size)
  DSP_Model.q = RangeSet(1, q_size)
  DSP_Model.v = RangeSet(1, v_size)
  DSP_Model.g = RangeSet(1, g_size)
  DSP_Model.k = RangeSet(1, k_size)
  DSP_Model.l = RangeSet(1, l_size)
  DSP_Model.p = RangeSet(1, p_size)
  DSP_Model.n = RangeSet(1, n_size)
  DSP_Model.m = RangeSet(1, m_size)
  DSP_Model.c = RangeSet(1, c_size)
  DSP_Model.r = RangeSet(1, r_size)
  DSP_Model.s = RangeSet(1, s_size)

  DSP_Model.fc = Param(DSP_Model.h, DSP_Model.j, initialize=parameters['fc'])
  DSP_Model.ff = Param(DSP_Model.v, DSP_Model.q, initialize=parameters['ff'])
  DSP_Model.fd = Param(DSP_Model.m, DSP_Model.n, initialize=parameters['fd'])
  DSP_Model.v1 = Param(DSP_Model.i, DSP_Model.j, initialize=parameters['v1'])
  DSP_Model.v2 = Param(DSP_Model.j, DSP_Model.q, initialize=parameters['v2'])
  DSP_Model.v3 = Param(DSP_Model.q, DSP_Model.k, initialize=parameters['v3'])
  DSP_Model.v4 = Param(DSP_Model.g, DSP_Model.k, initialize=parameters['v4'])
  DSP_Model.v5 = Param(DSP_Model.p, DSP_Model.k, DSP_Model.n, initialize=parameters['v5'])
  DSP_Model.v6 = Param(DSP_Model.p, DSP_Model.n, DSP_Model.c, initialize=parameters['v6'])
  DSP_Model.v7 = Param(DSP_Model.p, DSP_Model.k, DSP_Model.l, initialize=parameters['v7'])
  DSP_Model.cc = Param(DSP_Model.p, DSP_Model.k, initialize=parameters['cc'])
  DSP_Model.cp = Param(DSP_Model.i, initialize=parameters['cp'])
  DSP_Model.cq = Param(DSP_Model.g, initialize=parameters['cq'])
  DSP_Model.cr = Param(DSP_Model.n, DSP_Model.p, initialize=parameters['cr'])
  DSP_Model.ca1 = Param(DSP_Model.h, DSP_Model.j, initialize=parameters['ca1'])
  DSP_Model.ca2 = Param(DSP_Model.v, DSP_Model.q, initialize=parameters['ca2'])
  DSP_Model.ca3 = Param(DSP_Model.m, DSP_Model.n, initialize=parameters['ca3'])
  DSP_Model.ca4 = Param(DSP_Model.k, DSP_Model.p, initialize=parameters['ca4'])
  DSP_Model.t = Param(DSP_Model.i, DSP_Model.j, initialize=parameters['t'])
  DSP_Model.wp = Param(DSP_Model.k, initialize=parameters['wp'])
  DSP_Model.bp = Param(DSP_Model.p, initialize=parameters['bp'])
  DSP_Model.wb = Param(DSP_Model.j, initialize=parameters['wb'])
  DSP_Model.cs = Param(initialize=parameters['cs'])
  DSP_Model.cf = Param(initialize=parameters['cf'])
  DSP_Model.bs_hat = Param(DSP_Model.i, DSP_Model.s, initialize=parameters['bs_hat'])
  DSP_Model.bs_upper = Param(DSP_Model.i, initialize=parameters['bs_upper'])
  DSP_Model.bs_lower = Param(DSP_Model.i, initialize=parameters['bs_lower'])
  DSP_Model.ps = Param(DSP_Model.j, initialize=parameters['ps'])
  DSP_Model.pd = Param(DSP_Model.p, DSP_Model.c, initialize=parameters['pd'])
  DSP_Model.pe = Param(DSP_Model.p, DSP_Model.l, initialize=parameters['pe'])
  DSP_Model.dd_hat = Param(DSP_Model.p, DSP_Model.c, DSP_Model.s, initialize=parameters['dd_hat'])
  DSP_Model.dd_upper = Param(DSP_Model.p, DSP_Model.c, initialize=parameters['dd_upper'])
  DSP_Model.dd_lower = Param(DSP_Model.p, DSP_Model.c, initialize=parameters['dd_lower'])
  DSP_Model.de_hat = Param(DSP_Model.p, DSP_Model.l, DSP_Model.s, initialize=parameters['de_hat'])
  DSP_Model.de_upper = Param(DSP_Model.p, DSP_Model.l, initialize=parameters['de_upper'])
  DSP_Model.de_lower = Param(DSP_Model.p, DSP_Model.l, initialize=parameters['de_lower'])
  DSP_Model.b = Param(DSP_Model.r, DSP_Model.i, DSP_Model.j, initialize=parameters['b'])
  DSP_Model.g_b = Param(DSP_Model.r, DSP_Model.i, DSP_Model.j, initialize=parameters['g_b'])
  DSP_Model.Lambda = Param(DSP_Model.i, DSP_Model.j, initialize=parameters['Lambda'])
  DSP_Model.BigM = Param(initialize=parameters['BigM'])

  DSP_Model.X = Param(DSP_Model.h, DSP_Model.j, initialize=X_dict)
  DSP_Model.Y = Param(DSP_Model.v, DSP_Model.q, initialize=Y_dict)
  DSP_Model.Z = Param(DSP_Model.m, DSP_Model.n, initialize=Z_dict)

  DSP_Model.landa = landa
  DSP_Model.theta = theta


  DSP_Model.rho = Var(DSP_Model.i, within=NonNegativeReals)                           # ρ
  DSP_Model.vrho = Var(DSP_Model.p, DSP_Model.c, within=NonNegativeReals)             # ϱ
  DSP_Model.vsigma = Var(DSP_Model.p, DSP_Model.l, within=NonNegativeReals)           # ς
  DSP_Model.iota = Var(DSP_Model.j, within=NonNegativeReals)                          # ι
  DSP_Model.vphi = Var(DSP_Model.r, DSP_Model.i, DSP_Model.j, within=NonNegativeReals) # φ
  DSP_Model.gamma = Var(DSP_Model.i, DSP_Model.j, within=NonNegativeReals)            # γ
  DSP_Model.zeta = Var(DSP_Model.j, within=NonNegativeReals)                          # ζ
  DSP_Model.vtheta = Var(DSP_Model.q, within=NonNegativeReals)                        # ϑ
  DSP_Model.eta = Var(DSP_Model.n, within=NonNegativeReals)                           # η
  DSP_Model.kappa = Var(DSP_Model.j, within=NonNegativeReals)                         # κ
  DSP_Model.omicron = Var(DSP_Model.k, DSP_Model.p, within=NonNegativeReals)
  DSP_Model.mu = Var(DSP_Model.q, within=NonNegativeReals)                            # μ
  DSP_Model.nu = Var(DSP_Model.k, within=NonNegativeReals)                            # ν
  DSP_Model.vpi = Var(DSP_Model.n, DSP_Model.p, within=NonNegativeReals)             # ϖ
  DSP_Model.tau = Var(DSP_Model.i, DSP_Model.j, within=Reals)                         # τ
  DSP_Model.omega_plus = Var(DSP_Model.i, within=NonNegativeReals)                    # Ω⁺ᵢ
  DSP_Model.omega_minus = Var(DSP_Model.i, within=NonNegativeReals)                   # Ω⁻ᵢ
  DSP_Model.pi_plus = Var(DSP_Model.p, DSP_Model.c, within=NonNegativeReals)
  DSP_Model.pi_minus = Var(DSP_Model.p, DSP_Model.c, within=NonNegativeReals)
  DSP_Model.Delta_plus = Var(DSP_Model.p, DSP_Model.l, within=NonNegativeReals)       # Δ⁺ₚₗ
  DSP_Model.Delta_minus = Var(DSP_Model.p, DSP_Model.l, within=NonNegativeReals)      # Δ⁻ₚₗ
  DSP_Model.delta_plus = Var(DSP_Model.i, within=Binary)
  DSP_Model.delta_minus = Var(DSP_Model.i, within=Binary)
  DSP_Model.chi_plus = Var(DSP_Model.p, DSP_Model.c, within=Binary)                   # χ⁺ₚ꜀
  DSP_Model.chi_minus = Var(DSP_Model.p, DSP_Model.c, within=Binary)                  # χ⁻ₚ꜀
  DSP_Model.Gamma_plus = Var(DSP_Model.p, DSP_Model.l, within=Binary)                 # Γ⁺ₚₗ
  DSP_Model.Gamma_minus = Var(DSP_Model.p, DSP_Model.l, within=Binary)                # Γ⁻ₚₗ

  obj = -1*(
    sum(DSP_Model.gamma[i,j]*DSP_Model.BigM*DSP_Model.t[i,j] for i in DSP_Model.i for j in DSP_Model.j)
    + sum(DSP_Model.zeta[j]*sum(DSP_Model.ca1[h,j]*DSP_Model.X[h,j] for h in DSP_Model.h) for j in DSP_Model.j)
    + sum(DSP_Model.vtheta[q]*sum(DSP_Model.ca2[v,q]*DSP_Model.Y[v,q] for v in DSP_Model.v) for q in DSP_Model.q)
    + sum(DSP_Model.eta[n]*sum(DSP_Model.ca3[m,n]*DSP_Model.Z[m,n] for m in DSP_Model.m) for n in DSP_Model.n)
    + sum(DSP_Model.omicron[k,p]*DSP_Model.ca4[k,p] for k in DSP_Model.k for p in DSP_Model.p)
    + sum((DSP_Model.bs_upper[i]-DSP_Model.bs_hat[i,ss])*DSP_Model.omega_plus[i] + (DSP_Model.bs_lower[i]-DSP_Model.bs_hat[i,ss])*DSP_Model.omega_minus[i] + DSP_Model.bs_hat[i,ss]*DSP_Model.rho[i] for i in DSP_Model.i)
    + sum((DSP_Model.dd_upper[p,c]-DSP_Model.dd_hat[p,c,ss])*DSP_Model.pi_plus[p,c] + (DSP_Model.dd_lower[p,c]-DSP_Model.dd_hat[p,c,ss])*DSP_Model.pi_minus[p,c] + DSP_Model.dd_hat[p,c,ss]*DSP_Model.vrho[p,c] for p in DSP_Model.p for c in DSP_Model.c)
    + sum((DSP_Model.de_upper[p,l]-DSP_Model.de_hat[p,l,ss])*DSP_Model.Delta_plus[p,l] + (DSP_Model.de_lower[p,l]-DSP_Model.de_hat[p,l,ss])*DSP_Model.Delta_minus[p,l] + DSP_Model.de_hat[p,l,ss]*DSP_Model.vsigma[p,l] for p in DSP_Model.p for l in DSP_Model.l)
    - sum(DSP_Model.tau[i,j]*sum(DSP_Model.X[h,j] for h in DSP_Model.h) for i in DSP_Model.i for j in DSP_Model.j)
    + sum(DSP_Model.vphi[r,i,j] for r in DSP_Model.r for i in DSP_Model.i for j in DSP_Model.j)
    + sum(DSP_Model.landa*((DSP_Model.bs_upper[i]-DSP_Model.bs_hat[i,ss])*DSP_Model.delta_plus[i] + (DSP_Model.bs_hat[i,ss]-DSP_Model.bs_lower[i])*DSP_Model.delta_minus[i]) for i in DSP_Model.i)
    + sum(DSP_Model.landa*((DSP_Model.dd_upper[p,c]-DSP_Model.dd_hat[p,c,ss])*DSP_Model.chi_plus[p,c] + (DSP_Model.dd_hat[p,c,ss]-DSP_Model.dd_lower[p,c])*DSP_Model.chi_minus[p,c]) for p in DSP_Model.p for c in DSP_Model.c)
    + sum(DSP_Model.landa*((DSP_Model.de_upper[p,l]-DSP_Model.de_hat[p,l,ss])*DSP_Model.Gamma_plus[p,l] + (DSP_Model.de_hat[p,l,ss]-DSP_Model.de_lower[p,l])*DSP_Model.Gamma_minus[p,l]) for p in DSP_Model.p for l in DSP_Model.l)
    )
  DSP_Model.objective = Objective(sense=maximize, expr=obj)

  DSP_Model.cons1 = ConstraintList()
  for i in DSP_Model.i:
    for j in DSP_Model.j:
      lhs = -1*DSP_Model.gamma[i,j] - DSP_Model.zeta[j] + DSP_Model.wb[j]*DSP_Model.cs*DSP_Model.iota[j] + DSP_Model.wb[j]*(1-DSP_Model.cs)*DSP_Model.kappa[j] - DSP_Model.rho[i]
      rhs = DSP_Model.v1[i,j] + DSP_Model.cp[i]
      DSP_Model.cons1.add(lhs <= rhs)

  DSP_Model.cons2 = ConstraintList()
  for j in DSP_Model.j:
    for q in DSP_Model.q:
      lhs = -1*DSP_Model.vtheta[q] - DSP_Model.iota[j] + DSP_Model.cf*DSP_Model.mu[q]
      rhs = DSP_Model.v2[j,q]
      DSP_Model.cons2.add(lhs <= rhs)

  DSP_Model.cons3 = ConstraintList()
  for j in DSP_Model.j:
    lhs = DSP_Model.kappa[j]
    rhs = DSP_Model.ps[j]
    DSP_Model.cons3.add(lhs >= rhs)

  DSP_Model.cons4 = ConstraintList()
  for q in DSP_Model.q:
    for k in DSP_Model.k:
      lhs = -1*DSP_Model.mu[q] + DSP_Model.wp[k]*DSP_Model.nu[k]
      rhs = DSP_Model.v3[q,k]
      DSP_Model.cons4.add(lhs <= rhs)

  DSP_Model.cons5 = ConstraintList()
  for g in DSP_Model.g:
    for k in DSP_Model.k:
      lhs = DSP_Model.wp[k]*DSP_Model.nu[k]
      rhs = DSP_Model.v4[g,k] + DSP_Model.cq[g]
      DSP_Model.cons5.add(lhs <= rhs)

  DSP_Model.cons6 = ConstraintList()
  for p in DSP_Model.p:
    for k in DSP_Model.k:
      for n in DSP_Model.n:
        lhs = -1*DSP_Model.eta[n] - DSP_Model.bp[p]*DSP_Model.nu[k] - DSP_Model.omicron[k,p] + DSP_Model.vpi[n,p]
        rhs = DSP_Model.v5[p,k,n] + DSP_Model.cc[p,k] + DSP_Model.cr[n,p]
        DSP_Model.cons6.add(lhs <= rhs)

  DSP_Model.cons7 = ConstraintList()
  for p in DSP_Model.p:
    for n in DSP_Model.n:
      for c in DSP_Model.c:
        lhs = -1*DSP_Model.vpi[n,p] - DSP_Model.vrho[p,c]
        rhs = DSP_Model.v6[p,n,c] - DSP_Model.pd[p,c]
        DSP_Model.cons7.add(lhs <= rhs)

  DSP_Model.cons8 = ConstraintList()
  for p in DSP_Model.p:
    for k in DSP_Model.k:
      for l in DSP_Model.l:
        lhs = -1*DSP_Model.bp[p]*DSP_Model.nu[k] - DSP_Model.omicron[k,p] - DSP_Model.vsigma[p,l]
        rhs = DSP_Model.v7[p,k,l] + DSP_Model.cc[p,k] - DSP_Model.pe[p,l]
        DSP_Model.cons8.add(lhs <= rhs)

  DSP_Model.cons9 = ConstraintList()
  for r in DSP_Model.r:
    for i in DSP_Model.i:
      for j in DSP_Model.j:
        lhs = DSP_Model.wb[j]*DSP_Model.Lambda[i,j]*DSP_Model.cs*DSP_Model.g_b[r,i,j]*DSP_Model.iota[j] + DSP_Model.wb[j]*DSP_Model.Lambda[i,j]*(1-DSP_Model.cs)*DSP_Model.g_b[r,i,j]*DSP_Model.kappa[j] + DSP_Model.tau[i,j] - DSP_Model.vphi[r,i,j]
        rhs = DSP_Model.b[r,i,j]
        DSP_Model.cons9.add(lhs <= rhs)

  DSP_Model.cons10 = ConstraintList()
  for i in DSP_Model.i:
    lhs = DSP_Model.omega_plus[i]
    rhs = DSP_Model.BigM * DSP_Model.delta_plus[i]
    DSP_Model.cons10.add(lhs <= rhs)

  DSP_Model.cons11 = ConstraintList()
  for i in DSP_Model.i:
    lhs = DSP_Model.omega_plus[i]
    rhs = DSP_Model.rho[i]
    DSP_Model.cons11.add(lhs <= rhs)

  DSP_Model.cons12 = ConstraintList()
  for i in DSP_Model.i:
    lhs = DSP_Model.omega_plus[i]
    rhs = DSP_Model.rho[i] - DSP_Model.BigM * (1 - DSP_Model.delta_plus[i])
    DSP_Model.cons12.add(lhs >= rhs)

  DSP_Model.cons13 = ConstraintList()
  for i in DSP_Model.i:
    lhs = DSP_Model.omega_minus[i]
    rhs = DSP_Model.BigM * DSP_Model.delta_minus[i]
    DSP_Model.cons13.add(lhs <= rhs)

  DSP_Model.cons14 = ConstraintList()
  for i in DSP_Model.i:
    lhs = DSP_Model.omega_minus[i]
    rhs = DSP_Model.rho[i]
    DSP_Model.cons14.add(lhs <= rhs)

  DSP_Model.cons15 = ConstraintList()
  for i in DSP_Model.i:
    lhs = DSP_Model.omega_minus[i]
    rhs = DSP_Model.rho[i] - DSP_Model.BigM * (1 - DSP_Model.delta_minus[i])
    DSP_Model.cons15.add(lhs >= rhs)

  DSP_Model.cons16 = ConstraintList()
  for i in DSP_Model.i:
    lhs = DSP_Model.omega_plus[i] + DSP_Model.omega_minus[i]
    rhs = 1
    DSP_Model.cons16.add(lhs <= rhs)

  DSP_Model.cons17 = ConstraintList()
  for p in DSP_Model.p:
    for c in DSP_Model.c:
      lhs = DSP_Model.pi_plus[p,c]
      rhs = DSP_Model.BigM * DSP_Model.chi_plus[p,c]
      DSP_Model.cons17.add(lhs <= rhs)

  DSP_Model.cons18 = ConstraintList()
  for p in DSP_Model.p:
    for c in DSP_Model.c:
      lhs = DSP_Model.pi_plus[p,c]
      rhs = DSP_Model.vrho[p,c]
      DSP_Model.cons18.add(lhs <= rhs)

  DSP_Model.cons19 = ConstraintList()
  for p in DSP_Model.p:
    for c in DSP_Model.c:
      lhs = DSP_Model.pi_plus[p,c]
      rhs = DSP_Model.vrho[p,c] - DSP_Model.BigM * (1 - DSP_Model.chi_plus[p,c])
      DSP_Model.cons19.add(lhs >= rhs)

  DSP_Model.cons20 = ConstraintList()
  for p in DSP_Model.p:
    for c in DSP_Model.c:
      lhs = DSP_Model.pi_minus[p,c]
      rhs = DSP_Model.BigM * DSP_Model.chi_minus[p,c]
      DSP_Model.cons20.add(lhs <= rhs)

  DSP_Model.cons21 = ConstraintList()
  for p in DSP_Model.p:
    for c in DSP_Model.c:
      lhs = DSP_Model.pi_minus[p,c]
      rhs = DSP_Model.vrho[p,c]
      DSP_Model.cons21.add(lhs <= rhs)

  DSP_Model.cons22 = ConstraintList()
  for p in DSP_Model.p:
    for c in DSP_Model.c:
      lhs = DSP_Model.pi_minus[p,c]
      rhs = DSP_Model.vrho[p,c] - DSP_Model.BigM * (1 - DSP_Model.chi_minus[p,c])
      DSP_Model.cons22.add(lhs >= rhs)

  DSP_Model.cons23 = ConstraintList()
  for p in DSP_Model.p:
    for c in DSP_Model.c:
      lhs = DSP_Model.chi_plus[p,c] + DSP_Model.chi_minus[p,c]
      rhs = 1
      DSP_Model.cons23.add(lhs <= rhs)

  DSP_Model.cons24 = ConstraintList()
  for p in DSP_Model.p:
    for l in DSP_Model.l:
      lhs = DSP_Model.Delta_plus[p,l]
      rhs = DSP_Model.BigM * DSP_Model.Gamma_plus[p,l]
      DSP_Model.cons24.add(lhs <= rhs)

  DSP_Model.cons25 = ConstraintList()
  for p in DSP_Model.p:
    for l in DSP_Model.l:
      lhs = DSP_Model.Delta_plus[p,l]
      rhs = DSP_Model.vsigma[p,l]
      DSP_Model.cons25.add(lhs <= rhs)

  DSP_Model.cons26 = ConstraintList()
  for p in DSP_Model.p:
    for l in DSP_Model.l:
      lhs = DSP_Model.Delta_plus[p,l]
      rhs = DSP_Model.vsigma[p,l] - DSP_Model.BigM * (1 - DSP_Model.Gamma_plus[p,l])
      DSP_Model.cons26.add(lhs >= rhs)

  DSP_Model.cons27 = ConstraintList()
  for p in DSP_Model.p:
    for l in DSP_Model.l:
      lhs = DSP_Model.Delta_minus[p,l]
      rhs = DSP_Model.BigM * DSP_Model.Gamma_minus[p,l]
      DSP_Model.cons27.add(lhs <= rhs)

  DSP_Model.cons28 = ConstraintList()
  for p in DSP_Model.p:
    for l in DSP_Model.l:
      lhs = DSP_Model.Delta_minus[p,l]
      rhs = DSP_Model.vsigma[p,l]
      DSP_Model.cons28.add(lhs <= rhs)

  DSP_Model.cons29 = ConstraintList()
  for p in DSP_Model.p:
    for l in DSP_Model.l:
      lhs = DSP_Model.Delta_minus[p,l]
      rhs = DSP_Model.vsigma[p,l] - DSP_Model.BigM * (1 - DSP_Model.Gamma_minus[p,l])
      DSP_Model.cons29.add(lhs >= rhs)

  DSP_Model.cons30 = ConstraintList()
  for p in DSP_Model.p:
    for l in DSP_Model.l:
      lhs = DSP_Model.Gamma_plus[p,l] + DSP_Model.Gamma_minus[p,l]
      rhs = 1
      DSP_Model.cons30.add(lhs <= rhs)

  if theta == 0:
    DSP_Model.cons31 = ConstraintList()
    for i in DSP_Model.i:
      lhs = DSP_Model.delta_plus[i]
      rhs = 0
      DSP_Model.cons31.add(lhs == rhs)

    DSP_Model.cons32 = ConstraintList()
    for i in DSP_Model.i:
      lhs = DSP_Model.delta_minus[i]
      rhs = 0
      DSP_Model.cons32.add(lhs == rhs)

    DSP_Model.cons33 = ConstraintList()
    for p in DSP_Model.p:
      for c in DSP_Model.c:
        lhs = DSP_Model.chi_plus[p,c]
        rhs = 0
        DSP_Model.cons33.add(lhs == rhs)

    DSP_Model.cons34 = ConstraintList()
    for p in DSP_Model.p:
      for c in DSP_Model.c:
        lhs = DSP_Model.chi_minus[p,c]
        rhs = 0
        DSP_Model.cons34.add(lhs == rhs)

    DSP_Model.cons35 = ConstraintList()
    for p in DSP_Model.p:
      for l in DSP_Model.l:
        lhs = DSP_Model.Gamma_plus[p,l]
        rhs = 0
        DSP_Model.cons35.add(lhs == rhs)

    DSP_Model.cons36 = ConstraintList()
    for p in DSP_Model.p:
      for l in DSP_Model.l:
        lhs = DSP_Model.Gamma_minus[p,l]
        rhs = 0
        DSP_Model.cons36.add(lhs == rhs)


  # Solver setup
  solver = SolverFactory('glpk')
  results_DSP_Model = solver.solve(DSP_Model)

  DSP_Value = pyomo.environ.value(DSP_Model.objective)

  Output_Variabels = {
      'rho': {(i, ss, it): DSP_Model.rho[i].value for i in DSP_Model.i},
      'vrho': {(p, c, ss, it): DSP_Model.vrho[p, c].value for p in DSP_Model.p for c in DSP_Model.c},
      'vsigma': {(p, l, ss, it): DSP_Model.vsigma[p, l].value for p in DSP_Model.p for l in DSP_Model.l},
      'iota': {(j, ss, it): DSP_Model.iota[j].value for j in DSP_Model.j},
      'vphi': {(r, i, j, ss, it): DSP_Model.vphi[r, i, j].value for r in DSP_Model.r for i in DSP_Model.i for j in DSP_Model.j},
      'gamma': {(i, j, ss, it): DSP_Model.gamma[i, j].value for i in DSP_Model.i for j in DSP_Model.j},
      'zeta': {(j, ss, it): DSP_Model.zeta[j].value for j in DSP_Model.j},
      'vtheta': {(q, ss, it): DSP_Model.vtheta[q].value for q in DSP_Model.q},
      'eta': {(n, ss, it): DSP_Model.eta[n].value for n in DSP_Model.n},
      'kappa': {(j, ss, it): DSP_Model.kappa[j].value for j in DSP_Model.j},
      'omicron': {(k, p, ss, it): DSP_Model.omicron[k, p].value for k in DSP_Model.k for p in DSP_Model.p},
      'mu': {(q, ss, it): DSP_Model.mu[q].value for q in DSP_Model.q},
      'nu': {(k, ss, it): DSP_Model.nu[k].value for k in DSP_Model.k},
      'vpi': {(n, p, ss, it): DSP_Model.vpi[n, p].value for n in DSP_Model.n for p in DSP_Model.p},
      'tau': {(i, j, ss, it): DSP_Model.tau[i, j].value for i in DSP_Model.i for j in DSP_Model.j},
      'omega_plus': {(i, ss, it): DSP_Model.omega_plus[i].value for i in DSP_Model.i},
      'omega_minus': {(i, ss, it): DSP_Model.omega_minus[i].value for i in DSP_Model.i},
      'pi_plus': {(p, c, ss, it): DSP_Model.pi_plus[p, c].value for p in DSP_Model.p for c in DSP_Model.c},
      'pi_minus': {(p, c, ss, it): DSP_Model.pi_minus[p, c].value for p in DSP_Model.p for c in DSP_Model.c},
      'Delta_plus': {(p, l, ss, it): DSP_Model.Delta_plus[p, l].value for p in DSP_Model.p for l in DSP_Model.l},
      'Delta_minus': {(p, l, ss, it): DSP_Model.Delta_minus[p, l].value for p in DSP_Model.p for l in DSP_Model.l},
      'delta_plus': {(i, ss, it): DSP_Model.delta_plus[i].value for i in DSP_Model.i},
      'delta_minus': {(i, ss, it): DSP_Model.delta_minus[i].value for i in DSP_Model.i},
      'chi_plus': {(p, c, ss, it): DSP_Model.chi_plus[p, c].value for p in DSP_Model.p for c in DSP_Model.c},
      'chi_minus': {(p, c, ss, it): DSP_Model.chi_minus[p, c].value for p in DSP_Model.p for c in DSP_Model.c},
      'Gamma_plus': {(p, l, ss, it): DSP_Model.Gamma_plus[p, l].value for p in DSP_Model.p for l in DSP_Model.l},
      'Gamma_minus': {(p, l, ss, it): DSP_Model.Gamma_minus[p, l].value for p in DSP_Model.p for l in DSP_Model.l},
  }

  return [Output_Variabels , DSP_Value]

In [ ]:
def RMP(Fixd_Variabels,parameters,theta,it):
  global i_size, j_size, h_size, q_size, v_size, g_size, k_size, l_size, p_size, n_size, m_size, c_size, r_size, s_size
  RMP_Model = ConcreteModel()

  RMP_Model.i = RangeSet(1, i_size)
  RMP_Model.j = RangeSet(1, j_size)
  RMP_Model.h = RangeSet(1, h_size)
  RMP_Model.q = RangeSet(1, q_size)
  RMP_Model.v = RangeSet(1, v_size)
  RMP_Model.g = RangeSet(1, g_size)
  RMP_Model.k = RangeSet(1, k_size)
  RMP_Model.l = RangeSet(1, l_size)
  RMP_Model.p = RangeSet(1, p_size)
  RMP_Model.n = RangeSet(1, n_size)
  RMP_Model.m = RangeSet(1, m_size)
  RMP_Model.c = RangeSet(1, c_size)
  RMP_Model.r = RangeSet(1, r_size)
  RMP_Model.s = RangeSet(1, s_size)
  RMP_Model.it = RangeSet(1, it)


  RMP_Model.fc = Param(RMP_Model.h, RMP_Model.j, initialize=parameters['fc'])
  RMP_Model.ff = Param(RMP_Model.v, RMP_Model.q, initialize=parameters['ff'])
  RMP_Model.fd = Param(RMP_Model.m, RMP_Model.n, initialize=parameters['fd'])
  RMP_Model.v1 = Param(RMP_Model.i, RMP_Model.j, initialize=parameters['v1'])
  RMP_Model.v2 = Param(RMP_Model.j, RMP_Model.q, initialize=parameters['v2'])
  RMP_Model.v3 = Param(RMP_Model.q, RMP_Model.k, initialize=parameters['v3'])
  RMP_Model.v4 = Param(RMP_Model.g, RMP_Model.k, initialize=parameters['v4'])
  RMP_Model.v5 = Param(RMP_Model.p, RMP_Model.k, RMP_Model.n, initialize=parameters['v5'])
  RMP_Model.v6 = Param(RMP_Model.p, RMP_Model.n, RMP_Model.c, initialize=parameters['v6'])
  RMP_Model.v7 = Param(RMP_Model.p, RMP_Model.k, RMP_Model.l, initialize=parameters['v7'])
  RMP_Model.cc = Param(RMP_Model.p, RMP_Model.k, initialize=parameters['cc'])
  RMP_Model.cp = Param(RMP_Model.i, initialize=parameters['cp'])
  RMP_Model.cq = Param(RMP_Model.g, initialize=parameters['cq'])
  RMP_Model.cr = Param(RMP_Model.n, RMP_Model.p, initialize=parameters['cr'])
  RMP_Model.ca1 = Param(RMP_Model.h, RMP_Model.j, initialize=parameters['ca1'])
  RMP_Model.ca2 = Param(RMP_Model.v, RMP_Model.q, initialize=parameters['ca2'])
  RMP_Model.ca3 = Param(RMP_Model.m, RMP_Model.n, initialize=parameters['ca3'])
  RMP_Model.ca4 = Param(RMP_Model.k, RMP_Model.p, initialize=parameters['ca4'])
  RMP_Model.t = Param(RMP_Model.i, RMP_Model.j, initialize=parameters['t'])
  RMP_Model.wp = Param(RMP_Model.k, initialize=parameters['wp'])
  RMP_Model.bp = Param(RMP_Model.p, initialize=parameters['bp'])
  RMP_Model.wb = Param(RMP_Model.j, initialize=parameters['wb'])
  RMP_Model.cs = Param(initialize=parameters['cs'])
  RMP_Model.cf = Param(initialize=parameters['cf'])
  RMP_Model.bs_hat = Param(RMP_Model.i, RMP_Model.s, initialize=parameters['bs_hat'])
  RMP_Model.bs_upper = Param(RMP_Model.i, initialize=parameters['bs_upper'])
  RMP_Model.bs_lower = Param(RMP_Model.i, initialize=parameters['bs_lower'])
  RMP_Model.ps = Param(RMP_Model.j, initialize=parameters['ps'])
  RMP_Model.pd = Param(RMP_Model.p, RMP_Model.c, initialize=parameters['pd'])
  RMP_Model.pe = Param(RMP_Model.p, RMP_Model.l, initialize=parameters['pe'])
  RMP_Model.dd_hat = Param(RMP_Model.p, RMP_Model.c, RMP_Model.s, initialize=parameters['dd_hat'])
  RMP_Model.dd_upper = Param(RMP_Model.p, RMP_Model.c, initialize=parameters['dd_upper'])
  RMP_Model.dd_lower = Param(RMP_Model.p, RMP_Model.c, initialize=parameters['dd_lower'])
  RMP_Model.de_hat = Param(RMP_Model.p, RMP_Model.l, RMP_Model.s, initialize=parameters['de_hat'])
  RMP_Model.de_upper = Param(RMP_Model.p, RMP_Model.l, initialize=parameters['de_upper'])
  RMP_Model.de_lower = Param(RMP_Model.p, RMP_Model.l, initialize=parameters['de_lower'])
  RMP_Model.b = Param(RMP_Model.r, RMP_Model.i, RMP_Model.j, initialize=parameters['b'])
  RMP_Model.g_b = Param(RMP_Model.r, RMP_Model.i, RMP_Model.j, initialize=parameters['g_b'])
  RMP_Model.Lambda = Param(RMP_Model.i, RMP_Model.j, initialize=parameters['Lambda'])
  RMP_Model.BigM = Param(initialize=parameters['BigM'])

  RMP_Model.rho_star = Param(RMP_Model.i, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['rho'])
  RMP_Model.vrho_star = Param(RMP_Model.p, RMP_Model.c, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['vrho'])
  RMP_Model.vsigma_star = Param(RMP_Model.p, RMP_Model.l, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['vsigma'])
  RMP_Model.iota_star = Param(RMP_Model.j, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['iota'])
  RMP_Model.vphi_star = Param(RMP_Model.r, RMP_Model.i, RMP_Model.j, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['vphi'])
  RMP_Model.gamma_star = Param(RMP_Model.i, RMP_Model.j, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['gamma'])
  RMP_Model.zeta_star = Param(RMP_Model.j, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['zeta'])
  RMP_Model.vtheta_star = Param(RMP_Model.q, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['vtheta'])
  RMP_Model.eta_star = Param(RMP_Model.n, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['eta'])
  RMP_Model.kappa_star = Param(RMP_Model.j, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['kappa'])
  RMP_Model.omicron_star = Param(RMP_Model.k, RMP_Model.p, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['omicron'])
  RMP_Model.mu_star = Param(RMP_Model.q, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['mu'])
  RMP_Model.nu_star = Param(RMP_Model.k, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['nu'])
  RMP_Model.vpi_star = Param(RMP_Model.n, RMP_Model.p, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['vpi'])
  RMP_Model.tau_star = Param(RMP_Model.i, RMP_Model.j, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['tau'])
  RMP_Model.omega_plus_star = Param(RMP_Model.i, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['omega_plus'])
  RMP_Model.omega_minus_star = Param(RMP_Model.i, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['omega_minus'])
  RMP_Model.pi_plus_star = Param(RMP_Model.p, RMP_Model.c, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['pi_plus'])
  RMP_Model.pi_minus_star = Param(RMP_Model.p, RMP_Model.c, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['pi_minus'])
  RMP_Model.Delta_plus_star = Param(RMP_Model.p, RMP_Model.l, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['Delta_plus'])
  RMP_Model.Delta_minus_star = Param(RMP_Model.p, RMP_Model.l, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['Delta_minus'])
  RMP_Model.delta_plus_star = Param(RMP_Model.i, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['delta_plus'])
  RMP_Model.delta_minus_star = Param(RMP_Model.i, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['delta_minus'])
  RMP_Model.chi_plus_star = Param(RMP_Model.p, RMP_Model.c, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['chi_plus'])
  RMP_Model.chi_minus_star = Param(RMP_Model.p, RMP_Model.c, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['chi_minus'])
  RMP_Model.Gamma_plus_star = Param(RMP_Model.p, RMP_Model.l, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['Gamma_plus'])
  RMP_Model.Gamma_minus_star = Param(RMP_Model.p, RMP_Model.l, RMP_Model.s, RMP_Model.it, initialize=Fixd_Variabels['Gamma_minus'])

  RMP_Model.theta = theta

  RMP_Model.X = Var(RMP_Model.h, RMP_Model.j, within=Binary)
  RMP_Model.Y = Var(RMP_Model.v, RMP_Model.q, within=Binary)
  RMP_Model.Z = Var(RMP_Model.m, RMP_Model.n, within=Binary)
  RMP_Model.landa = Var(within=NonNegativeReals)
  RMP_Model.phi = Var(RMP_Model.s, within=Reals)

  obj = (sum(RMP_Model.fc[h,j]*RMP_Model.X[h,j] for h in RMP_Model.h for j in RMP_Model.j)
  + sum(RMP_Model.ff[v,q]*RMP_Model.Y[v,q] for v in RMP_Model.v for q in RMP_Model.q)
  + sum(RMP_Model.fd[m,n]*RMP_Model.Z[m,n] for m in RMP_Model.m for n in RMP_Model.n)
  + RMP_Model.theta*RMP_Model.landa
  + (1/s_size)*sum(RMP_Model.phi[s] for s in RMP_Model.s)
         )
  RMP_Model.objective = Objective(sense=minimize, expr=obj)

  RMP_Model.cons1 = Constraint(expr=sum(RMP_Model.X[h,j] for h in RMP_Model.h for j in RMP_Model.j) >= 1)

  RMP_Model.cons2 = Constraint(expr=sum(RMP_Model.Y[v,q] for v in RMP_Model.v for q in RMP_Model.q) >= 1)

  RMP_Model.cons3 = Constraint(expr=sum(RMP_Model.Z[m,n] for m in RMP_Model.m for n in RMP_Model.n) >= 1)

  RMP_Model.cons4 = ConstraintList()
  for j in RMP_Model.j:
      lhs = sum(RMP_Model.X[h,j] for h in RMP_Model.h)
      rhs = 1
      RMP_Model.cons4.add(lhs <= rhs)

  RMP_Model.cons5 = ConstraintList()
  for q in RMP_Model.q:
      lhs = sum(RMP_Model.Y[v,q] for v in RMP_Model.v)
      rhs = 1
      RMP_Model.cons5.add(lhs <= rhs)

  RMP_Model.cons6 = ConstraintList()
  for n in RMP_Model.n:
      lhs = sum(RMP_Model.Z[m,n] for m in RMP_Model.m)
      rhs = 1
      RMP_Model.cons6.add(lhs <= rhs)

  RMP_Model.cons7 = ConstraintList()
  for ss in RMP_Model.s:
    for itt in RMP_Model.it:
      lhs = (RMP_Model.phi[ss] + sum(RMP_Model.gamma_star[i,j,ss,itt]*RMP_Model.BigM*RMP_Model.t[i,j] for i in RMP_Model.i for j in RMP_Model.j)
      + sum(RMP_Model.zeta_star[j,ss,itt]*sum(RMP_Model.ca1[h,j]*RMP_Model.X[h,j] for h in RMP_Model.h) for j in RMP_Model.j)
      + sum(RMP_Model.vtheta_star[q,ss,itt]*sum(RMP_Model.ca2[v,q]*RMP_Model.Y[v,q] for v in RMP_Model.v) for q in RMP_Model.q)
      + sum(RMP_Model.eta_star[n,ss,itt]*sum(RMP_Model.ca3[m,n]*RMP_Model.Z[m,n] for m in RMP_Model.m) for n in RMP_Model.n)
      + sum(RMP_Model.omicron_star[k,p,ss,itt]*RMP_Model.ca4[k,p] for k in RMP_Model.k for p in RMP_Model.p)
      + sum((RMP_Model.bs_upper[i]-RMP_Model.bs_hat[i,ss])*RMP_Model.omega_plus_star[i,ss,itt] + (RMP_Model.bs_lower[i]-RMP_Model.bs_hat[i,ss])*RMP_Model.omega_minus_star[i,ss,itt] + RMP_Model.bs_hat[i,ss]*RMP_Model.rho_star[i,ss,itt] for i in RMP_Model.i)
      + sum((RMP_Model.dd_upper[p,c]-RMP_Model.dd_hat[p,c,ss])*RMP_Model.pi_plus_star[p,c,ss,itt] + (RMP_Model.dd_lower[p,c]-RMP_Model.dd_hat[p,c,ss])*RMP_Model.pi_minus_star[p,c,ss,itt] + RMP_Model.dd_hat[p,c,ss]*RMP_Model.vrho_star[p,c,ss,itt] for p in RMP_Model.p for c in RMP_Model.c)
      + sum((RMP_Model.de_upper[p,l]-RMP_Model.de_hat[p,l,ss])*RMP_Model.Delta_plus_star[p,l,ss,itt] + (RMP_Model.de_lower[p,l]-RMP_Model.de_hat[p,l,ss])*RMP_Model.Delta_minus_star[p,l,ss,itt] + RMP_Model.de_hat[p,l,ss]*RMP_Model.vsigma_star[p,l,ss,itt] for p in RMP_Model.p for l in RMP_Model.l)
      + sum(RMP_Model.vphi_star[r,i,j,ss,itt] for r in RMP_Model.r for i in RMP_Model.i for j in RMP_Model.j)
      + sum(RMP_Model.landa*((RMP_Model.bs_upper[i]-RMP_Model.bs_hat[i,ss])*RMP_Model.delta_plus_star[i,ss,itt] + (RMP_Model.bs_hat[i,ss]-RMP_Model.bs_lower[i])*RMP_Model.delta_minus_star[i,ss,itt]) for i in RMP_Model.i)
      + sum(RMP_Model.landa*((RMP_Model.dd_upper[p,c]-RMP_Model.dd_hat[p,c,ss])*RMP_Model.chi_plus_star[p,c,ss,itt] + (RMP_Model.dd_hat[p,c,ss]-RMP_Model.dd_lower[p,c])*RMP_Model.chi_minus_star[p,c,ss,itt]) for p in RMP_Model.p for c in RMP_Model.c)
      + sum(RMP_Model.landa*((RMP_Model.de_upper[p,l]-RMP_Model.de_hat[p,l,ss])*RMP_Model.Gamma_plus_star[p,l,ss,itt] + (RMP_Model.de_hat[p,l,ss]-RMP_Model.de_lower[p,l])*RMP_Model.Gamma_minus_star[p,l,ss,itt]) for p in RMP_Model.p for l in RMP_Model.l)
      )
      rhs = sum(RMP_Model.tau_star[i,j,ss,itt]*sum(RMP_Model.X[h,j] for h in RMP_Model.h) for i in RMP_Model.i for j in RMP_Model.j)
      RMP_Model.cons7.add(lhs >= rhs)

  RMP_Model.cons8 = Constraint(expr=RMP_Model.landa <= 100000)

  # if 'cons8' in RMP_Model.component_map(Constraint):
  #     RMP_Model.del_component('cons8')

  # if theta == 0:
  #     RMP_Model.cons8 = Constraint(expr=RMP_Model.landa <= 100000)

  # Solver setup
  solver = SolverFactory('glpk')
  results_RMP_Model = solver.solve(RMP_Model)

  RMP_Value = pyomo.environ.value(RMP_Model.objective)

  X_value = {(h,j): RMP_Model.X[h,j].value for h in RMP_Model.h for j in RMP_Model.j}
  Y_value = {(v,q): RMP_Model.Y[v,q].value for v in RMP_Model.v for q in RMP_Model.q}
  Z_value = {(m,n): RMP_Model.Z[m,n].value for m in RMP_Model.m for n in RMP_Model.n}
  landa_value = RMP_Model.landa.value


  return [X_value,Y_value,Z_value,landa_value,RMP_Value]

In [ ]:
import time

# X = {}
# for h in Model.h:
#     for j in Model.j:
#         if h == 1:
#             X[(h, j)] = 1
#         else:
#             X[(h, j)] = 0

# Y = {}
# for v in Model.v:
#     for q in Model.q:
#         if v == 1:
#             Y[(v, q)] = 1
#         else:
#             Y[(v, q)] = 0


# Z = {}
# for m in Model.m:
#     for n in Model.n:
#         if m == 1:
#             Z[(m, n)] = 1
#         else:
#             Z[(m, n)] = 0


X = {(1, 1): 0.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 0.0, (1, 5): 0.0, (1, 6): 0.0, (1, 7): 0.0, (1, 8): 0.0, (1, 9): 0.0, (1, 10): 0.0, (1, 11): 0.0, (1, 12): 0.0, (1, 13): 0.0, (1, 14): 0.0, (1, 15): 0.0, (2, 1): 0.0, (2, 2): 1.0, (2, 3): 0.0, (2, 4): 0.0, (2, 5): 1.0, (2, 6): 0.0, (2, 7): 0.0, (2, 8): 0.0, (2, 9): 0.0, (2, 10): 0.0, (2, 11): 0.0, (2, 12): 0.0, (2, 13): 0.0, (2, 14): 0.0, (2, 15): 0.0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0.0, (3, 4): 0.0, (3, 5): 0.0, (3, 6): 1.0, (3, 7): 0.0, (3, 8): 0.0, (3, 9): 0.0, (3, 10): 0.0, (3, 11): 0.0, (3, 12): 0.0, (3, 13): 0.0, (3, 14): 0.0, (3, 15): 0.0}
Y = {(1, 1): 0.0, (1, 2): 0.0, (1, 3): 1.0, (1, 4): 0.0, (1, 5): 0.0, (1, 6): 0.0, (1, 7): 0.0, (1, 8): 0.0, (1, 9): 1.0, (1, 10): 1.0, (2, 1): 0.0, (2, 2): 0.0, (2, 3): 0.0, (2, 4): 0.0, (2, 5): 0.0, (2, 6): 0.0, (2, 7): 0.0, (2, 8): 0.0, (2, 9): 0.0, (2, 10): 0.0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0.0, (3, 4): 0.0, (3, 5): 1.0, (3, 6): 0.0, (3, 7): 0.0, (3, 8): 1.0, (3, 9): 0.0, (3, 10): 0.0}
Z = {(1, 1): 0.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 0.0, (1, 5): 0.0, (1, 6): 0.0, (1, 7): 0.0, (1, 8): 0.0, (1, 9): 0.0, (1, 10): 0.0, (1, 11): 0.0, (1, 12): 0.0, (1, 13): 0.0, (1, 14): 0.0, (1, 15): 0.0, (1, 16): 0.0, (1, 17): 0.0, (1, 18): 0.0, (1, 19): 0.0, (1, 20): 0.0, (2, 1): 0.0, (2, 2): 0.0, (2, 3): 0.0, (2, 4): 0.0, (2, 5): 0.0, (2, 6): 0.0, (2, 7): 0.0, (2, 8): 0.0, (2, 9): 0.0, (2, 10): 0.0, (2, 11): 0.0, (2, 12): 1.0, (2, 13): 0.0, (2, 14): 0.0, (2, 15): 0.0, (2, 16): 0.0, (2, 17): 0.0, (2, 18): 0.0, (2, 19): 0.0, (2, 20): 0.0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0.0, (3, 4): 0.0, (3, 5): 0.0, (3, 6): 0.0, (3, 7): 0.0, (3, 8): 0.0, (3, 9): 0.0, (3, 10): 0.0, (3, 11): 0.0, (3, 12): 0.0, (3, 13): 0.0, (3, 14): 0.0, (3, 15): 0.0, (3, 16): 0.0, (3, 17): 0.0, (3, 18): 0.0, (3, 19): 0.0, (3, 20): 0.0}

landa = 20

Fixd_Variabels = {
    'rho': {},
    'vrho': {},
    'vsigma': {},
    'iota': {},
    'vphi': {},
    'gamma': {},
    'zeta': {},
    'vtheta': {},
    'eta': {},
    'kappa': {},
    'omicron': {},
    'mu': {},
    'nu': {},
    'vpi': {},
    'tau': {},
    'omega_plus': {},
    'omega_minus': {},
    'pi_plus': {},
    'pi_minus': {},
    'Delta_plus': {},
    'Delta_minus': {},
    'delta_plus': {},
    'delta_minus': {},
    'chi_plus': {},
    'chi_minus': {},
    'Gamma_plus': {},
    'Gamma_minus': {},
}


theta = 200
UB = 0
Privous_UB = 0
start=time.time()
for iter in range(1, 100):
    etasum = 0
    for samp in range(1, s_size+1):
        Output_Variabels, dsp_value = DSP(parameters, X, Y, Z, landa, theta, samp, iter)
        etasum +=  dsp_value
        for key, value in Fixd_Variabels.items():
            Fixd_Variabels[key].update(Output_Variabels[key])

    UB += landa*theta + ((1/s_size)*etasum) + sum(fc[h,j]*X[h,j] for h in Model.h for j in Model.j) + sum(ff[v,q]*Y[v,q] for v in Model.v for q in Model.q) + sum(fd[m,n]*Z[m,n] for m in Model.m for n in Model.n)
    Privous_UB = min(UB,Privous_UB)
    print(f"UB for iteration {iter}: {Privous_UB}")
    X, Y, Z, landa, LB = RMP(Fixd_Variabels, parameters, theta, iter)
    print(f"LB for iteration {iter}: {LB}")
    print(f"Gap for iteration {iter}: {Privous_UB - LB}")
    if Privous_UB - LB < 0.01:
      break
    UB = 0
    print(40*"-")
end=time.time()
print("Run time: ",end-start)

UB for iteration 1: 0
LB for iteration 1: -1606.4916629351387
Gap for iteration 1: 1606.4916629351387
----------------------------------------
UB for iteration 2: -1023.7325157703317
LB for iteration 2: -1398.5373203526333
Gap for iteration 2: 374.8048045823016
----------------------------------------
UB for iteration 3: -1023.7325157703317
LB for iteration 3: -1375.2631842288981
Gap for iteration 3: 351.5306684585664
----------------------------------------
UB for iteration 4: -1153.6082866226625
LB for iteration 4: -1279.5142699685616
Gap for iteration 4: 125.90598334589913
----------------------------------------
UB for iteration 5: -1153.6082866226625
LB for iteration 5: -1267.6631984172805
Gap for iteration 5: 114.05491179461796
----------------------------------------
UB for iteration 6: -1209.1699966356787
LB for iteration 6: -1250.9487245501577
Gap for iteration 6: 41.77872791447908
----------------------------------------
UB for iteration 7: -1209.1699966356787
LB for iteratio

In [ ]:
Z

In [ ]:
for h in Model.h:
  for j in Model.j:
    if X[(h,j)] == 1:
      print(f'X[{h},{j}] = 1')
print(40*"-")

for v in Model.v:
  for q in Model.q:
    if Y[(v,q)] == 1:
      print(f'Y[{v},{q}] = 1')
print(40*"-")

for m in Model.m:
  for n in Model.n:
    if Z[(m,n)] == 1:
      print(f'Z[{m},{n}] = 1')
